### Step 1. Download input data

In [1]:
# !rm -rf *
# !mkdir example_gtfs_bart
# !mkdir outputs
# !mkdir outputs/train_outputs
# !mkdir outputs/traveler_outputs

# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/example/example_gtfs_bart/stop_times.txt" -O example_gtfs_bart/stop_times.txt
# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/example/example_gtfs_bart/stops.txt" -O example_gtfs_bart/stops.txt
# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/example/example_gtfs_bart/trips.txt" -O example_gtfs_bart/trips.txt

# !mkdir model
# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/main/model/transit_sim_model.py" -O model/transit_sim_model.py

# !mkdir sp
# !wget "https://github.com/UCB-CE170a/Fall2021/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
# !wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2021/master/traffic_data/interface.py" -O sp/interface.py

# !pip install geopandas

### Step 2. Import modules

In [2]:
### plotting
import matplotlib.pyplot as plt

### user module
from model.transit_sim_model import Network, Trains, Travelers

### fix random seed
import numpy as np
np.random.seed(0)

### Step 3. Process GTFS schedules
Need the following tables:
 * stop_times.txt: schedule info
 * trips.txt: map trip_id to route_id
 * stops.txt: get stop coordinates (for visualization)

In [3]:
### read in GTFS files
stop_times_file = '../uregina/inputs/gtfs_changping_stop_times.csv'
trips_file = '../uregina/inputs/gtfs_changping_trips.csv'
stops_file = '../uregina/inputs/gtfs_changping_stops.csv'
### only keep results with this service id
service_id = 'weekday'

### create all trains from GTFS
all_trains = Trains()
all_nodes, all_links = all_trains.schedule_and_network_from_gtfs(
    stop_times_file, trips_file, stops_file, service_id)

### create network from nodes and links
network = Network(all_nodes, all_links)

In [4]:
### display and export network
display(network.all_nodes.head(3))
display(network.all_links.head(2))
network.all_links.to_csv('../uregina/outputs/changping_links.csv', index=False)
network.all_nodes.to_csv('../uregina/outputs/changping_nodes.csv', index=False)

,route_stop_id,stop_lon,stop_lat,stop_id,type,node_id,geometry
0,downward-changping_xishankou,1124.75,-450.75,changping_xishankou,platform,0,POINT (1124.750 -450.750)
1,downward-shisanling_jingqu,1235.75,-465.75,shisanling_jingqu,platform,1,POINT (1235.750 -465.750)
2,downward-changping,1367.75,-533.25,changping,platform,2,POINT (1367.750 -533.250)


,route_stop_id,next_route_stop_id,start_nid,end_nid,initial_weight,geometry
2835,downward-changping_xishankou,downward-shisanling_jingqu,0,1,1.0,"LINESTRING (1124.750 -450.750, 1235.750 -465.750)"
2836,downward-shisanling_jingqu,downward-changping,1,2,1.0,"LINESTRING (1235.750 -465.750, 1367.750 -533.250)"


### Step 4. Travel demand
 * random demand
 * or, input csv with columns *traveler_id*, *origin_nid*, *destin_nid*, *departure_time*

In [5]:
import pandas as pd 
travelers_df = pd.read_csv('../uregina/inputs/changping_line_test_od.csv')

station_to_node_dict = {getattr(row, 'stop_id'): getattr(row, 'node_id') for row in network.all_nodes.itertuples()}
travelers_df['origin_nid'] = travelers_df['enter_station'].map(station_to_node_dict).astype(int)
travelers_df['destin_nid'] = travelers_df['exit_station'].map(station_to_node_dict).astype(int)
travelers_df['traveler_id'] = np.arange(travelers_df.shape[0])
travelers_df['departure_time'] = travelers_df['enter_hour']*3600 + travelers_df[
    'enter_minute']*60 + travelers_df['enter_second'] + np.random.randint(0, 60, size=travelers_df.shape[0])
display(travelers_df.iloc[6])

travelers = Travelers()
travelers.travelers_df = travelers_df[['traveler_id', 'origin_nid', 'destin_nid', 'departure_time']].copy()
travelers.find_routes(network.network_g, network.station_id_nm_dict)
travelers.set_initial_status(network.station_id_nm_dict)

display(travelers.travelers_df.head())

ENTRY_TIME              2018/4/1 5:59:00
DEAL_TIME               2018/4/1 6:15:54
TRIP_ORIGIN_LOCATION           151019041
CURRENT_LOCATION               151019045
enter_hour                             5
enter_minute                          59
enter_second                           0
arrival_hour                           6
arrival_minute                        15
arrival_second                        54
enter_station                      shahe
exit_station                zhuxinzhuang
origin_nid                            30
destin_nid                            35
traveler_id                            6
departure_time                     21543
Name: 6, dtype: object

,traveler_id,origin_nid,destin_nid,departure_time,traveler_status,update_time,association,next_station,origin_station,destination_station
0,0,34,30,21344,pretrip,0,None,None,all-xierqi,all-shahe
1,1,31,29,20507,pretrip,0,None,None,all-shahe_gaojiaoyuan,all-nanshao
2,2,35,30,20873,pretrip,0,None,None,all-zhuxinzhuang,all-shahe
3,3,34,30,20400,pretrip,0,None,None,all-xierqi,all-shahe
4,4,32,31,19983,pretrip,0,None,None,all-shengming_kexueyuan,all-shahe_gaojiaoyuan


In [6]:
# ### random OD
# ### create instance
# travelers = Travelers()
# ### generate random od
# travelers.random_od(network.all_nodes, num_travelers=100)
# ### find routes
# travelers.find_routes(network.network_g, network.station_id_nm_dict)
# travelers.set_initial_status(network.station_id_nm_dict)

# display(travelers.travelers_df)

In [7]:
### check if travelers can transfer between lines
#print(travelers.travelers_paths[80])
print(travelers.travelers_key_stops[6])
# all_trains.schedule_df[(all_trains.schedule_df['location']=='1-COLM') & 
#                       (all_trains.schedule_df['next_time']>29400+140) &
#                        (all_trains.schedule_df['next_time']<29424+1000)
#                       ]
# all_trains.schedule_df[all_trains.schedule_df['trip_id']==964621].head()
display(travelers.travelers_df.iloc[6:7])
print(travelers.travelers_df.shape)
print(np.min(travelers.travelers_df['departure_time']), np.max(travelers.travelers_df['departure_time']))

{'all-shahe': 'downward-shahe', 'downward-shahe': 'downward-zhuxinzhuang', 'downward-zhuxinzhuang': 'all-zhuxinzhuang'}


,traveler_id,origin_nid,destin_nid,departure_time,traveler_status,update_time,association,next_station,origin_station,destination_station
6,6,30,35,21543,pretrip,0,None,None,all-shahe,all-zhuxinzhuang


(8205, 10)
18001 21599


### Step 5. Run the simulation

In [8]:
def save_agg_results(network, trains, travelers, t):
    ### save train results
    train_positions = trains.get_all_train_positions(network)
    train_positions.to_csv('../uregina/outputs/train_outputs/train_outputs_cptest_{}.csv'.format(t), index=False)
    ### save aggregated traveler results
    traveler_locations = travelers.travelers_df.groupby(
            ['traveler_status', 'association']).size().to_frame(
            name='num_travelers').reset_index(drop=False)
    traveler_locations.to_csv('../uregina/outputs/traveler_outputs/agg_traveler_outputs_cptest_{}.csv'.format(t), 
                              index=False)

def save_indiv_results(network, trains, travelers, t):
    ### save individual traveler results
    travelers.travelers_df.to_csv('../uregina/outputs/traveler_outputs/indiv_traveler_outputs_cptest_{}.csv'.format(t), 
                                  index=False)

In [9]:
t_init, t_end = 18000, 18000+3600*2
for t in range(t_init, t_end, 20):
    #print(t)
    ### update train location
    all_trains.update_location_occupancy(t)
    
    ### update traveler status
    travelers.traveler_update(network, all_trains, t)
    
    ### print and plot results
    #plot_map(network, all_trains, travelers, t)
    save_agg_results(network, all_trains, travelers, t)
    
    if (t-t_init)%600==0:
        save_indiv_results(network, all_trains, travelers, t)
        print('Simulation at {}:{:<02}am'.format(t//3600, (t%3600)//60))

Simulation at 5:00am
Simulation at 5:10am
Simulation at 5:20am
Simulation at 5:30am
Simulation at 5:40am
      trip_id     time  next_time status                      location  \
746     22002  20711.0    20741.0   stop    upward-shengming_kexueyuan   
1844    51003  20685.0    20730.0   stop         downward-zhuxinzhuang   
3044   321007  20687.0    20732.0   stop  downward-changping_xishankou   

     current_location  
746           current  
1844          current  
3044          current   

      trip_id     time  next_time status                      location  \
746     22002  20711.0    20741.0   stop    upward-shengming_kexueyuan   
1844    51003  20685.0    20730.0   stop         downward-zhuxinzhuang   
3044   321007  20687.0    20732.0   stop  downward-changping_xishankou   

     current_location  train_occupancy  
746           current            204.0  
1844          current             75.0  
3044          current              0.0   

     trip_id     time  next_time stat

      trip_id     time  next_time status            location current_location
3048   321007  21072.0    21102.0   stop  downward-changping          current 

      trip_id     time  next_time status            location current_location  \
3048   321007  21072.0    21102.0   stop  downward-changping          current   

      train_occupancy  
3048              0.0   

      trip_id     time  next_time status                    location  \
3048   321007  21072.0    21102.0   stop          downward-changping   
7728    12801  21095.0    21125.0   stop  upward-changping_xishankou   

     current_location  
3048          current  
7728          current   

      trip_id     time  next_time status                    location  \
3048   321007  21072.0    21102.0   stop          downward-changping   
7728    12801  21095.0    21125.0   stop  upward-changping_xishankou   

     current_location  train_occupancy  
3048          current              0.0  
7728          current              6.0 

In [10]:
### zip outputs and upload to the transit_vis notebook
!tar czvf outputs.tar.gz outputs

tar: outputs: Cannot stat: No such file or directory
tar: Error exit delayed from previous errors.
